In [1]:
import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
import nfp

In [2]:
redox_model = tf.keras.models.load_model(
    '/projects/rlmolecule/pstjohn/models/20210107_redox_model/',
    compile=False)

In [3]:
def atom_featurizer(atom):
    """ Return an integer hash representing the atom type
    """

    return str((
        atom.GetSymbol(),
        atom.GetNumRadicalElectrons(),
        atom.GetFormalCharge(),
        atom.GetChiralTag().name,
        atom.GetIsAromatic(),
        nfp.get_ring_size(atom, max_size=6),
        atom.GetDegree(),
        atom.GetTotalNumHs(includeNeighbors=True)
    ))


def bond_featurizer(bond, flipped=False):

    if not flipped:
        atoms = "{}-{}".format(
            *tuple((bond.GetBeginAtom().GetSymbol(),
                    bond.GetEndAtom().GetSymbol())))
    else:
        atoms = "{}-{}".format(
            *tuple((bond.GetEndAtom().GetSymbol(),
                    bond.GetBeginAtom().GetSymbol())))

    bstereo = bond.GetStereo().name
    btype = str(bond.GetBondType())
    ring = 'R{}'.format(nfp.get_ring_size(bond, max_size=6)) if bond.IsInRing() else ''

    return " ".join([atoms, btype, ring, bstereo]).strip()


preprocessor = nfp.SmilesPreprocessor(
    atom_features=atom_featurizer, bond_features=bond_featurizer, explicit_hs=False)

preprocessor.from_json('/projects/rlmolecule/pstjohn/models/20210107_redox_model/preprocessor.json')

In [ ]:
preprocessor.

In [4]:
def pred_redox(smiles):
    return redox_model.predict(
        {key: tf.constant(np.expand_dims(val, 0))
         for key, val in preprocessor.construct_feature_matrices(smiles, train=False).items()}).flatten()

In [8]:
preprocessor.construct_feature_matrices('CC(=S)C1=C(C)[CH]CCC1(C)C', train=False)

{'n_atom': 12,
 'n_bond': 12,
 'bond_indices': array([ 0,  0,  1,  2,  1,  2,  3, 11,  3,  4,  5,  4,  5,  6,  6,  7,  7,
         8,  8,  9, 10, 11,  9, 10]),
 'atom': array([  8,   4, 107,  24,  24,   8,  27,  26,  26,  52,   8,   8]),
 'bond': array([ 3,  3, 70,  3, 71,  3, 41, 14, 41,  3, 14,  3, 14, 14, 14, 14, 14,
        14, 14,  3,  3, 14,  3,  3]),
 'connectivity': array([[ 0,  1],
        [ 1,  0],
        [ 1,  2],
        [ 1,  3],
        [ 2,  1],
        [ 3,  1],
        [ 3,  4],
        [ 3,  9],
        [ 4,  3],
        [ 4,  5],
        [ 4,  6],
        [ 5,  4],
        [ 6,  4],
        [ 6,  7],
        [ 7,  6],
        [ 7,  8],
        [ 8,  7],
        [ 8,  9],
        [ 9,  8],
        [ 9, 10],
        [ 9, 11],
        [ 9,  3],
        [10,  9],
        [11,  9]])}

In [5]:
pred_redox('CC(=S)C1=C(C)[CH]CCC1(C)C')

/projects/rlmolecule/pstjohn/envs/tf2_gpu/lib/python3.7/site-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['n_bond', 'bond_indices'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


array([ 0.71126366, -0.4947876 ], dtype=float32)

In [6]:
import pandas as pd
gng = pd.read_csv('20210109_gng_radicals.csv.gz', index_col=0)

In [7]:
gng[gng.smiles == 'CC(=S)C1=C(C)[CH]CCC1(C)C']

,smiles,time,real_reward,atom_type,buried_vol,max_spin,atom_index,ie,ea,bde
823300,CC(=S)C1=C(C)[CH]CCC1(C)C,2021-01-08 21:20:03.435049,181.28307,C,71.99054,0.332517,3,0.711264,-0.494788,79.90403
